# ML Quality Eval: Validate with Multi-Gas Emissions (MOVESTAR)
Evaluates ML model quality using:
- Traditional regression metrics (R², RMSE, MAE)
- **CO2, HC, CO Emissions per second using MOVESTAR**
- Visual comparison: Ground Truth vs Predicted emissions (separate plots per gas)

In [1]:
# CELL 1: Parameters
RUN_TIMESTAMP = "2025-01-01_00-00-00"
INPUT_TEST_DATA = "s3://models-quality-eval-ml/test/test_data.pkl"
INPUT_ML_MODEL_PATH = "s3://models-quality-eval-ml/models/speed_accel_model.pkl"
OUTPUT_METRICS_PATH = "s3://models-quality-eval-ml/metrics/quality_metrics.json"
OUTPUT_PLOT_PATH = "s3://models-quality-eval-ml/metrics/validation_plots.png"

# Emission output paths (one per gas type)
OUTPUT_CO2_PLOT_PATH = "s3://models-quality-eval-ml/metrics/emission_co2_comparison.png"
OUTPUT_HC_PLOT_PATH = "s3://models-quality-eval-ml/metrics/emission_hc_comparison.png"
OUTPUT_CO_PLOT_PATH = "s3://models-quality-eval-ml/metrics/emission_co_comparison.png"

VEHICLE_TYPE = 1  # 1=Motorcycle, 2=Car

# Quality Thresholds
MIN_R2_SCORE = 0.85
MAX_SPEED_RMSE = 2.5  # m/s
MAX_ACCEL_RMSE = 0.7  # m/s²
MAX_SPEED_MAE = 2.0   # m/s
MAX_ACCEL_MAE = 0.5   # m/s²
MAX_CO2_ERROR_PERCENT = 15.0
MAX_HC_ERROR_PERCENT = 20.0
MAX_CO_ERROR_PERCENT = 20.0


# VSP Quality Threshold (NEW!)
MAX_VSP_RMSE = 2.0    # kW/ton
VEHICLE_TYPE_VSP = 'motorcycle'  # 'motorcycle' or 'car' for VSP calculation

MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"

In [2]:
# Parameters
RUN_TIMESTAMP = "2025-12-07_07-13-20"
INPUT_TEST_DATA = "s3://models-quality-eval-ml/test/test_data.pkl"
INPUT_ML_MODEL_PATH = "s3://models-quality-eval-ml/models/speed_accel_model.pkl"
OUTPUT_METRICS_PATH = (
    "s3://models-quality-eval-ml/metrics/quality_metrics_with_emissions.json"
)
OUTPUT_PLOT_PATH = (
    "s3://models-quality-eval-ml/metrics/validation_plots_with_emissions.png"
)
MIN_R2_SCORE = 0.85
MAX_SPEED_RMSE = 2.5
MAX_ACCEL_RMSE = 0.7
MAX_SPEED_MAE = 2.0
MAX_ACCEL_MAE = 0.5
MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"


In [ ]:
# CELL 2: Imports
import pandas as pd
import numpy as np
import pickle
import json
import io
import s3fs
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

sns.set_style('whitegrid')
print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [4]:
# CELL 3: MinIO Configuration
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)

storage_options = {
    "key": MINIO_ACCESS_KEY,
    "secret": MINIO_SECRET_KEY,
    "client_kwargs": {"endpoint_url": MINIO_ENDPOINT}
}

print("✅ MinIO connection initialized")

✅ MinIO connection initialized


In [ ]:
# CELL 4: Physics & Helper Functions
def calculate_vsp(speed_ms, acc_ms2, veh_type=1):
    """Calculates VSP using MOVESTAR coefficients (kW/ton)."""
    # 1 = Motorcycle, else = Car
    if veh_type == 1:
        A, B, C, M, f = 0.0251, 0.0, 0.000315, 0.285, 0.285
    else:
        A, B, C, M, f = 0.156461, 0.002002, 0.000493, 1.4788, 1.4788
        
    vsp = (A * speed_ms) + (B * speed_ms**2) + (C * speed_ms**3) + (M * speed_ms * acc_ms2)
    return vsp / f

def get_distribution(data, bins):
    """Get normalized distribution from data for binning comparison."""
    counts, _ = np.histogram(data, bins=bins)
    total = np.sum(counts)
    return counts / total if total > 0 else np.zeros(len(counts))

def calculate_symmetric_kl_divergence(p, q, epsilon=1e-10):
    """
    Calculate Symmetric KL Divergence: d(P,Q) = [KL(P||Q) + KL(Q||P)] / 2
    Lower = Better match between distributions.
    """
    p = p + epsilon
    q = q + epsilon
    p = p / np.sum(p)
    q = q / np.sum(q)
    
    kl_pq = np.sum(p * np.log(p / q))
    kl_qp = np.sum(q * np.log(q / p))
    return (kl_pq + kl_qp) / 2.0

print("✅ Physics & Helper functions loaded!")

✅ MOVESTAR multi-gas emission functions defined


In [ ]:
# CELL 5: Load Test Data and Model
print(f"=== Load artifacts and timestamp ===")
print(f"Run Timestamp: {RUN_TIMESTAMP}")
print(f"Vehicle Type: {'Motorcycle' if VEHICLE_TYPE == 1 else 'Car'}")

# Load test data
try:
    with fs.open(INPUT_TEST_DATA, 'rb') as f:
        df_test = pickle.load(f)
    
    if isinstance(df_test, pd.DataFrame):
        print(f"✅ Loaded test DataFrame with {len(df_test):,} rows")
    else:
        raise TypeError(f"Expected DataFrame, got {type(df_test)}")
except FileNotFoundError:
    print(f"❌ Error: {INPUT_TEST_DATA} not found. Run step 01 first.")
    raise

# Load trained model
try:
    with fs.open(INPUT_ML_MODEL_PATH, 'rb') as f:
        model_artifact = pickle.load(f)
    
    scaler = model_artifact['scaler']
    speed_model = model_artifact['speed_model']
    feature_cols = model_artifact['feature_cols']
    model_name = model_artifact.get('speed_model_name', 'Unknown')
    train_metrics = model_artifact.get('train_metrics', {})
    
    print(f"✅ Model loaded: {model_name}")
    if train_metrics:
        print(f"   Training R²: {train_metrics.get('r2', 'N/A'):.4f}")
        print(f"   Training RMSE: {train_metrics.get('rmse', 'N/A'):.4f} m/s")
except FileNotFoundError:
    print(f"❌ Error: {INPUT_ML_MODEL_PATH} not found. Run step 03 first.")
    raise

=== ML Quality Validation with Multi-Gas Emissions ===
Run Timestamp: 2025-12-07_07-13-20
Vehicle Type: Motorcycle

Loading artifacts...


✅ Loaded test DataFrame with 2,591 rows


✅ Model loaded: XGBoost
   Training R²: 0.9995
   Training RMSE: 0.0683 m/s


In [ ]:
# CELL 6: Feature Engineering (Same as Training)
print("\nRename Column name")

# Column normalization
column_mapping = {
    'timestamp_sensor': 'timestamp',
    'latitude': 'position_lat',
    'longitude': 'position_long',
    'speed_ms': 'speed_mps',
    'altitude': 'enhanced_altitude',
    'acc_forward': 'acceleration_m_s2',
    'acceleration': 'acceleration'
}

for old, new in column_mapping.items():
    if old in df_test.columns and new not in df_test.columns:
        df_test.rename(columns={old: new}, inplace=True)


Performing feature engineering...


✅ Feature engineering complete


In [ ]:
# CELL 6: Advanced Feature Engineering (Test Set)
print("\nPerforming Advanced Feature Engineering on Test Set...")

if 'trip_id' in df_test.columns:
    df_test = df_test.sort_values(['trip_id', 'seconds_elapsed'])
else:
    df_test = df_test.sort_values('seconds_elapsed')

# Basic Features
grouper = df_test.groupby('trip_id')['speed_mps'] if 'trip_id' in df_test.columns else df_test['speed_mps']
df_test['speed_mps_prev1'] = grouper.shift(1).fillna(0)
df_test['speed_mps_prev2'] = grouper.shift(2).fillna(0)
df_test['accel_prev1'] = (df_test['speed_mps_prev1'] - df_test['speed_mps_prev2']).fillna(0)

# Rolling Features (Initial values, will be overwritten in loop)
roll_src = grouper.shift(1)
df_test['speed_roll_mean_5s'] = roll_src.rolling(5, min_periods=1).mean().fillna(0)
df_test['speed_roll_std_5s'] = roll_src.rolling(5, min_periods=1).std().fillna(0)

# Segment Context
if 'segment_id' in df_test.columns:
    df_test['segment_avg_speed'] = df_test.groupby('segment_id')['speed_mps'].transform('mean')
else:
    if 'trip_id' in df_test.columns:
        df_test['segment_avg_speed'] = df_test.groupby('trip_id')['speed_mps'].expanding().mean().reset_index(0, drop=True)
    else:
        df_test['segment_avg_speed'] = df_test['speed_mps'].expanding().mean()

# Map Features
if 'enhanced_altitude' in df_test.columns:
    grouper_alt = df_test.groupby('trip_id')['enhanced_altitude'] if 'trip_id' in df_test.columns else df_test['enhanced_altitude']
    df_test['elev_gain_m'] = grouper_alt.diff().fillna(0)
else:
    df_test['elev_gain_m'] = 0

if 'label_traffic' in df_test.columns:
    traffic_map = {'heavy': 2, 'moderate': 1, 'light': 0}
    df_test['traffic_level'] = df_test['label_traffic'].map(traffic_map).fillna(1)
else:
    df_test['traffic_level'] = 1 

if 'position_lat' in df_test.columns:
    if 'trip_id' in df_test.columns:
        df_test['delta_lat'] = df_test.groupby('trip_id')['position_lat'].diff().fillna(0)
        df_test['delta_lon'] = df_test.groupby('trip_id')['position_long'].diff().fillna(0)
    else:
        df_test['delta_lat'] = df_test['position_lat'].diff().fillna(0)
        df_test['delta_lon'] = df_test['position_long'].diff().fillna(0)
    df_test['delta_dist'] = np.sqrt(df_test['delta_lat']**2 + df_test['delta_lon']**2)
else:
    df_test['delta_lat']=0; df_test['delta_lon']=0; df_test['delta_dist']=0

if 'bearing' in df_test.columns:
    if 'trip_id' in df_test.columns:
        df_test['heading_change'] = df_test.groupby('trip_id')['bearing'].diff().fillna(0)
    else:
        df_test['heading_change'] = df_test['bearing'].diff().fillna(0)
    df_test['turn_count'] = (np.abs(df_test['heading_change']) > 15).astype(int)
else:
    df_test['heading_change']=0; df_test['turn_count']=0

df_test = df_test.fillna(0)
print("✅ Test features prepared.")

In [ ]:
# CELL 7: Advanced Prediction Loop (Dynamic Rolling Update)
print("\n=== PREDICTING (With Rolling Momentum) ===")
from tqdm import tqdm
import collections

# --- 1. DEFINISI FITUR (Harus sama dengan Training!) ---
# Asumsi Anda menggunakan Advanced Feature Engineering
feature_cols = [
    'speed_mps_prev1', 'speed_mps_prev2', 'accel_prev1',
    'speed_roll_mean_5s', 'speed_roll_std_5s', # <-- Fitur Penting
    'segment_avg_speed',                       # <-- Fitur Penting
    'enhanced_altitude', 'bearing', 
    'delta_dist', 'delta_lat', 'delta_lon',
    'elev_gain_m', 'traffic_level', 'heading_change', 'turn_count'
]

# Filter hanya kolom yang ada (Safety)
available_cols = [c for c in feature_cols if c in df_test.columns]
print(f"Features used: {available_cols}")

# --- 2. PREPARATION ---
# Window untuk menghitung rolling stats secara live
history_window = collections.deque(maxlen=5) 
predictions = []

# --- 3. PREDICTION LOOP ---
for i in tqdm(range(len(df_test)), desc="Predicting"):
    
    # Cek Trip Baru
    is_new_trip = (i == 0) or ('trip_id' in df_test.columns and df_test.at[i, 'trip_id'] != df_test.at[i-1, 'trip_id'])
    
    if is_new_trip:
        history_window.clear()
        # Warm Start (Ambil data asli t=0)
        start_val = df_test.at[i, 'speed_mps']
        history_window.append(start_val)
        
        current_prev1 = 0.0
        current_prev2 = 0.0
        
        pred_speed = start_val 
    else:
        # Ambil input dari prediksi sebelumnya
        current_prev1 = predictions[-1]
        current_prev2 = predictions[-2] if len(predictions) > 1 else current_prev1
        
        # Hitung Rolling Stats dari History Prediksi Sendiri
        current_roll_mean = np.mean(history_window)
        current_roll_std = np.std(history_window) if len(history_window) > 1 else 0.0
        
        # Update DataFrame Features (Feedback Loop)
        df_test.at[i, 'speed_mps_prev1'] = current_prev1
        df_test.at[i, 'speed_mps_prev2'] = current_prev2
        df_test.at[i, 'accel_prev1'] = current_prev1 - current_prev2
        
        # Update Rolling Features (PENTING!)
        if 'speed_roll_mean_5s' in df_test.columns:
            df_test.at[i, 'speed_roll_mean_5s'] = current_roll_mean
            df_test.at[i, 'speed_roll_std_5s'] = current_roll_std
        
        # Predict
        X_row = df_test.loc[i, available_cols].values.reshape(1, -1)
        
        # Handle Scaling
        # (Jika winner-nya XGBoost dan tadi pakai RAW data, scaler tidak perlu dipakai sebenarnya,
        #  tapi code ini aman karena X_scaled dipakai cuma kalau modelnya butuh)
        if model_name in ['SVR', 'ANN']:
             try: X_row = scaler.transform(X_row)
             except: pass
            
        pred_speed = speed_model.predict(X_row)[0]
        
        # Safety
        if pred_speed < 0: pred_speed = 0
        if pred_speed > 50: pred_speed = 50

    predictions.append(pred_speed)
    history_window.append(pred_speed)

df_test['predicted_speed'] = predictions

# Hitung Accel
if 'trip_id' in df_test.columns:
    df_test['predicted_accel'] = df_test.groupby('trip_id')['predicted_speed'].diff().fillna(0)
else:
    df_test['predicted_accel'] = df_test['predicted_speed'].diff().fillna(0)

print(f"✅ Prediction Complete. Avg: {np.mean(predictions):.2f} m/s")

In [ ]:
# === HITUNG PREDICTED ACCEL ===
print("Calculating predicted acceleration...")

if 'trip_id' in df_test.columns:
    # Hitung selisih per trip (agar tidak bocor antar rute)
    df_test['predicted_accel'] = df_test.groupby('trip_id')['predicted_speed'].diff().fillna(0)
else:
    # Hitung selisih langsung (jika cuma 1 rute panjang)
    df_test['predicted_accel'] = df_test['predicted_speed'].diff().fillna(0)

# Verifikasi hasil
print(f"✅ Prediction Complete.")
print(f"   Avg Speed: {df_test['predicted_speed'].mean():.4f} m/s")
print(f"   Avg Accel: {df_test['predicted_accel'].mean():.4f} m/s²")

In [ ]:
# CELL 8: Metrics Helper Functions
def aggregate_metrics(metrics_dict):
    """
    Aggregate per-traffic metrics into combined overall metrics using weighted averaging.
    Matches the Markov Chain evaluation logic.
    """
    # Define groups to aggregate
    groups = [k for k in metrics_dict.keys() if k in ["Heavy Traffic", "Light Traffic"]]
    
    total_size = sum(metrics_dict[g]["test_sample_size_sec"] for g in groups)
    if total_size == 0: return {}
    
    aggregated = {
        "total_sample_size_sec": total_size,
        "validation_status": "PENDING",
        "failures": []
    }
    
    # List of metrics to average
    keys_to_average = [
        "avg_speed_real_kmh", "avg_speed_synthetic_kmh", "speed_difference_kmh",
        "avg_accel_real_ms2", "avg_accel_synthetic_ms2", "accel_difference_ms2",
        "std_speed_real_kmh", "std_speed_synthetic_kmh",
        "std_accel_real_ms2", "std_accel_synthetic_ms2",
        "vsp_rmse",
        "speed_r2", "speed_mae_kmh", "speed_rmse_kmh", "speed_mape_percent", # New
        "accel_r2", "accel_mae_ms2", "accel_rmse_ms2" # New
    ]
    
    for key in keys_to_average:
        weighted_sum = 0
        for g in groups:
            weight = metrics_dict[g]["test_sample_size_sec"] / total_size
            weighted_sum += metrics_dict[g].get(key, 0) * weight
        aggregated[key] = weighted_sum
        
    return aggregated

print("✅ Aggregation functions loaded")


=== TRADITIONAL METRICS ===

📊 SPEED METRICS:
  R²: 0.8561
  RMSE: 1.4735 m/s (5.30 km/h)
  MAE: 0.8067 m/s (2.90 km/h)
  MAPE: 19255508460420415327223553453927606255616.00%

📊 ACCELERATION METRICS:
  R²: 0.0204
  RMSE: 0.4782 m/s²
  MAE: 0.2797 m/s²


In [ ]:
# CELL 9: Metrics Calculation with Bias Correction (Scaling)
print("\n=== GENERATING ENHANCED METRICS (With Bias Correction) ===")

from scipy.ndimage import gaussian_filter1d
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# 1. Tentukan Traffic Split (25 km/h)
SPEED_THRESH_MPS = 25.0 / 3.6
traffic_groups = {
    "Heavy Traffic": df_test[df_test['speed_mps'] <= SPEED_THRESH_MPS],
    "Light Traffic": df_test[df_test['speed_mps'] > SPEED_THRESH_MPS]
}

vsp_bins = np.arange(-20, 40, 2)
speed_bins = np.arange(0, 120, 5)
metrics = {}

for label, df_group in traffic_groups.items():
    print(f"\nProcessing {label} ({len(df_group)} samples)...")
    if len(df_group) < 10:
        print("   -> Skip (Not enough data)")
        continue

    # --- A. DATA PREPARATION (REAL) ---
    # Smooth data asli untuk mengurangi noise GPS yang merusak VSP
    raw_real_speed_ms = df_group['speed_mps'].values
    real_speed_ms = gaussian_filter1d(raw_real_speed_ms, sigma=2.0)
    real_speed_kmh = real_speed_ms * 3.6
    
    # Hitung Real Accel dari speed yang sudah halus
    real_acc_ms2 = np.gradient(real_speed_ms)

    # --- B. DATA PREPARATION (PREDICTED + BIAS CORRECTION) ---
    raw_pred_speed_ms = df_group['predicted_speed'].values
    
    # 1. Bias Correction yang Lebih Lembut (Soft Scaling)
    bias_factor = 1.0 # No Scaling
    
    # 2. Terapkan Scaling
    pred_speed_ms = raw_pred_speed_ms * bias_factor
    pred_speed_kmh = pred_speed_ms * 3.6
    
    # 3. Hitung Accel dari Speed yang sudah dikoreksi
    pred_acc_ms2 = np.gradient(pred_speed_ms)
    
    print(f"   -> Applied Bias Correction: x{bias_factor:.2f}")

    # --- C. HITUNG VSP ---
    vsp_real = calculate_vsp(real_speed_ms, real_acc_ms2, VEHICLE_TYPE)
    vsp_pred = calculate_vsp(pred_speed_ms, pred_acc_ms2, VEHICLE_TYPE)

    # --- D. DISTRIBUSI ---
    speed_dist_real = get_distribution(real_speed_kmh, speed_bins)
    speed_dist_pred = get_distribution(pred_speed_kmh, speed_bins)
    vsp_dist_real = get_distribution(vsp_real, vsp_bins)
    vsp_dist_pred = get_distribution(vsp_pred, vsp_bins)

    # --- E. STATS & METRICS ---
    speed_rmse = np.sqrt(mean_squared_error(real_speed_kmh, pred_speed_kmh))
    speed_r2 = r2_score(real_speed_kmh, pred_speed_kmh)
    
    # Safety MAPE
    safe_real = np.where(real_speed_kmh < 1, 1, real_speed_kmh)
    speed_mape = np.mean(np.abs((real_speed_kmh - pred_speed_kmh) / safe_real)) * 100
    
    vsp_rmse_dist = np.sqrt(np.mean((vsp_dist_real - vsp_dist_pred)**2))
    
    # Store Metrics (PASTIKAN SEMUA KEY ADA)
    metrics[label] = {
        # -- Comparison Stats --
        "avg_speed_real_kmh": float(np.mean(real_speed_kmh)),
        "avg_speed_synthetic_kmh": float(np.mean(pred_speed_kmh)),
        "speed_difference_kmh": float(abs(np.mean(real_speed_kmh) - np.mean(pred_speed_kmh))),
        
        "avg_accel_real_ms2": float(np.mean(real_acc_ms2)),
        "avg_accel_synthetic_ms2": float(np.mean(pred_acc_ms2)),
        "accel_difference_ms2": float(abs(np.mean(real_acc_ms2) - np.mean(pred_acc_ms2))),
        
        # KEY YANG HILANG SEBELUMNYA (Sekarang sudah ada)
        "std_speed_real_kmh": float(np.std(real_speed_kmh)),
        "std_speed_synthetic_kmh": float(np.std(pred_speed_kmh)),
        "std_accel_real_ms2": float(np.std(real_acc_ms2)),
        "std_accel_synthetic_ms2": float(np.std(pred_acc_ms2)),
        
        "vsp_rmse": float(vsp_rmse_dist),
        "test_sample_size_sec": int(len(df_group)),

        # -- Regression Metrics --
        "speed_r2": float(speed_r2),
        "speed_rmse_kmh": float(speed_rmse),
        "speed_mape_percent": float(speed_mape),
        
        # -- Raw Data for Plotting --
        "_raw_data": {
            "speed_dist_real": speed_dist_real,
            "speed_dist_pred": speed_dist_pred,
            "vsp_dist_real": vsp_dist_real,
            "vsp_dist_pred": vsp_dist_pred
        }
    }

# Aggregate
final_output = {
    "overall": aggregate_metrics(metrics),
    "details": {k: {m: v for m, v in val.items() if not m.startswith('_')} for k, val in metrics.items()}
}

print(f"\n✅ Metrics Calculated. Overall VSP RMSE: {final_output['overall'].get('vsp_rmse', 0):.4f}")


=== MULTI-GAS EMISSION ANALYSIS (MOVESTAR) ===
🔥 Calculating CO2, HC, CO emissions...

🔥 CO2 RESULTS:
  Total Actual: 706.1500 g
  Total Pred: 742.4000 g
  Error: 5.13%
  Avg Rate (Actual): 0.272540 g/s
  Avg Rate (Pred): 0.286530 g/s
  RMSE: 0.129784 g/s
  MAE: 0.067406 g/s
  R²: 0.2095

🔥 HC RESULTS:
  Total Actual: 36.4110 g
  Total Pred: 36.6710 g
  Error: 0.71%
  Avg Rate (Actual): 0.014053 g/s
  Avg Rate (Pred): 0.014153 g/s
  RMSE: 0.005624 g/s
  MAE: 0.002448 g/s
  R²: -0.1606

🔥 CO RESULTS:
  Total Actual: 57.8740 g
  Total Pred: 58.6140 g
  Error: 1.28%
  Avg Rate (Actual): 0.022337 g/s
  Avg Rate (Pred): 0.022622 g/s
  RMSE: 0.008727 g/s
  MAE: 0.004164 g/s
  R²: -0.0788

✅ Emission analysis complete


In [ ]:
# CELL 10: Comparative Visualization (Grid Layout)
import matplotlib.pyplot as plt

# Setup layout: Rows = Traffic Groups, Cols = 4 (Speed Dist, VSP Dist, Speed Stats, Accel Stats)
groups_to_plot = [g for g in metrics.keys()]
n_rows = len(groups_to_plot)
fig = plt.figure(figsize=(20, 6 * n_rows)) # Perbesar tinggi agar lega

for idx, group_name in enumerate(groups_to_plot):
    data = metrics[group_name]
    raw = data["_raw_data"]
    row_base = idx * 4
    
    # 1. Speed Distribution
    ax1 = plt.subplot(n_rows, 4, row_base + 1)
    ax1.bar(speed_bins[:-1], raw["speed_dist_real"], width=4, alpha=0.5, label='Real', color='blue')
    ax1.plot(speed_bins[:-1], raw["speed_dist_pred"], color='red', linewidth=2, label='ML Pred')
    ax1.set_title(f"{group_name}\nSpeed Distribution\nR²={data['speed_r2']:.3f} | MAPE={data['speed_mape_percent']:.1f}%")
    ax1.set_xlabel("Speed (km/h)")
    ax1.set_ylabel("Probability")
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. VSP Distribution
    ax2 = plt.subplot(n_rows, 4, row_base + 2)
    ax2.bar(vsp_bins[:-1], raw["vsp_dist_real"], width=1.8, alpha=0.5, label='Real', color='blue')
    ax2.plot(vsp_bins[:-1], raw["vsp_dist_pred"], color='red', linewidth=2, label='ML Pred')
    ax2.set_title(f"VSP Distribution\nRMSE={data['vsp_rmse']:.3f}")
    ax2.set_xlabel("VSP (kW/ton)")
    ax2.set_ylabel("Probability")
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Speed Statistics (Bar Chart: Mean & Std)
    ax3 = plt.subplot(n_rows, 4, row_base + 3)
    categories = ['Mean', 'Std Dev']
    real_vals = [data['avg_speed_real_kmh'], data['std_speed_real_kmh']]
    pred_vals = [data['avg_speed_synthetic_kmh'], data['std_speed_synthetic_kmh']]
    x = np.arange(len(categories))
    width = 0.35
    
    ax3.bar(x - width/2, real_vals, width, label='Real', color='blue', alpha=0.7)
    ax3.bar(x + width/2, pred_vals, width, label='ML Pred', color='red', alpha=0.7)
    ax3.set_title(f"Speed Statistics\nΔ={data['speed_difference_kmh']:.2f} km/h")
    ax3.set_xticks(x)
    ax3.set_xticklabels(categories)
    ax3.set_ylabel("Speed (km/h)")
    ax3.legend()
    ax3.grid(True, axis='y', alpha=0.3)
    
    # 4. Accel Statistics (Bar Chart: Mean & Std)
    ax4 = plt.subplot(n_rows, 4, row_base + 4)
    real_vals = [data['avg_accel_real_ms2'], data['std_accel_real_ms2']]
    pred_vals = [data['avg_accel_synthetic_ms2'], data['std_accel_synthetic_ms2']]
    
    ax4.bar(x - width/2, real_vals, width, label='Real', color='blue', alpha=0.7)
    ax4.bar(x + width/2, pred_vals, width, label='ML Pred', color='red', alpha=0.7)
    ax4.set_title(f"Acceleration Statistics\nΔ={data['accel_difference_ms2']:.3f} m/s²")
    ax4.set_xticks(x)
    ax4.set_xticklabels(categories)
    ax4.set_ylabel("Accel (m/s²)")
    ax4.legend()
    ax4.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Save Plot
img_buf = io.BytesIO()
plt.savefig(img_buf, format='png', dpi=150)
img_buf.seek(0)
with fs.open(OUTPUT_PLOT_PATH, 'wb') as f:
    f.write(img_buf.getbuffer())
print(f"✅ Plots saved to {OUTPUT_PLOT_PATH}")


=== QUALITY EVALUATION (Unshifted) ===
{
  "model_name": "XGBoost",
  "run_timestamp": "2025-12-07_07-13-20",
  "test_samples": 2591,
  "vehicle_type": "Motorcycle",
  "acceleration_method": "speed_difference",
  "speed": {
    "r2_score": 0.8560842940947568,
    "rmse_ms": 1.4734899545952584,
    "rmse_kmh": 5.304563836542931,
    "mae_ms": 0.806690743475659,
    "mae_kmh": 2.9040866765123723,
    "mse": 2.171172646293137,
    "mape_percent": 1.9255508460420415e+40,
    "mean_actual": 4.602964962802235,
    "mean_predicted": 4.247651043947569,
    "std_actual": 3.884123973813831,
    "std_predicted": 3.253129449631045
  },
  "acceleration": {
    "r2_score": 0.020429042869914538,
    "rmse": 0.47821885627028654,
    "mae": 0.2797209972488795,
    "mse": 0.22869327449246096,
    "mape_percent": 102323.56545732707,
    "mean_actual": 0.001160211068148328,
    "mean_predicted": 0.0003055370438287544,
    "std_actual": 0.48317977386697547,
    "std_predicted": 0.5426394086522479
  },
  "


✅ Model Quality Validation Passed!

🎉 VALIDATION COMPLETE WITH MULTI-GAS EMISSIONS (Raw Predictions)
Model: XGBoost

Speed R²: 0.8561 | RMSE: 1.4735 m/s
Accel R²: 0.0204 | RMSE: 0.4782 m/s²

🔥 Raw Emission Errors:
  CO2: Error=5.1% | R²=0.209 ⚠️
  HC: Error=0.7% | R²=-0.161 ⚠️
  CO: Error=1.3% | R²=-0.079 ⚠️


In [ ]:
# CELL 10.5: Advanced Visualizations (Time Series & Heatmaps)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm

print("\n=== GENERATING ADVANCED VISUALIZATIONS ===")

# Ambil sample data dari salah satu traffic group yang ada isinya
target_group = "Light Traffic" if "Light Traffic" in metrics else "Heavy Traffic"
if target_group not in metrics:
    print("⚠️ No data available for visualization.")
else:
    df_viz = traffic_groups[target_group].copy()
    
    # Ambil 300 detik pertama untuk Time Series agar terlihat jelas
    limit = 300
    sample_df = df_viz.iloc[:limit]
    
    # Data Preparation
    t = range(len(sample_df))
    real_speed = sample_df['speed_mps'].values * 3.6
    pred_speed = sample_df['predicted_speed'].values * 3.6
    
    # Hitung VSP Detik-per-Detik (Second-by-Second VSP)
    # Real
    real_acc = np.gradient(sample_df['speed_mps'].values)
    vsp_real_ts = calculate_vsp(sample_df['speed_mps'].values, real_acc, VEHICLE_TYPE)
    
    # Pred
    pred_acc = np.gradient(sample_df['predicted_speed'].values)
    vsp_pred_ts = calculate_vsp(sample_df['predicted_speed'].values, pred_acc, VEHICLE_TYPE)

    # ==========================================
    # PLOT 1: TIME SERIES (Second per Second)
    # ==========================================
    fig, axes = plt.subplots(2, 1, figsize=(18, 10), sharex=True)
    
    # Speed Time Series
    axes[0].plot(t, real_speed, 'b-', label='Real Speed (Ground Truth)', alpha=0.7, linewidth=1.5)
    axes[0].plot(t, pred_speed, 'r--', label='ML Prediction', alpha=0.9, linewidth=1.5)
    axes[0].set_ylabel('Speed (km/h)', fontsize=12)
    axes[0].set_title(f'Second-by-Second Prediction: SPEED ({target_group})', fontsize=14, fontweight='bold')
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)
    
    # VSP Time Series (Based on Binning Logic)
    axes[1].plot(t, vsp_real_ts, 'b-', label='Real VSP', alpha=0.6, linewidth=1)
    axes[1].plot(t, vsp_pred_ts, 'r-', label='Predicted VSP', alpha=0.6, linewidth=1)
    axes[1].set_ylabel('VSP (kW/ton)', fontsize=12)
    axes[1].set_xlabel('Time (seconds)', fontsize=12)
    axes[1].set_title(f'Second-by-Second Prediction: VSP / POWER ({target_group})', fontsize=14, fontweight='bold')
    axes[1].legend(loc='upper right')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # ==========================================
    # PLOT 2: SPEED-ACCELERATION HEATMAPS (Binning Structure)
    # ==========================================
    # Ini membandingkan "bentuk" data secara keseluruhan berdasarkan binning
    
    fig2, axes2 = plt.subplots(1, 2, figsize=(18, 7))
    
    # Setup Bins
    x_bins = np.linspace(0, 80, 40) # Speed bins (0-80 km/h)
    y_bins = np.linspace(-3, 3, 40) # Accel bins (-3 to 3 m/s^2)
    
    # Real Heatmap
    h1 = axes2[0].hist2d(real_speed_kmh, real_acc_ms2, bins=[x_bins, y_bins], cmap='Blues', norm=LogNorm())
    axes2[0].set_title(f"REAL Data: Speed vs Accel Distribution", fontsize=14)
    axes2[0].set_xlabel("Speed (km/h)")
    axes2[0].set_ylabel("Acceleration (m/s²)")
    plt.colorbar(h1[3], ax=axes2[0], label='Log Frequency')
    
    # Predicted Heatmap
    h2 = axes2[1].hist2d(pred_speed_kmh, pred_acc_ms2, bins=[x_bins, y_bins], cmap='Reds', norm=LogNorm())
    axes2[1].set_title(f"PREDICTED Data: Speed vs Accel Distribution", fontsize=14)
    axes2[1].set_xlabel("Speed (km/h)")
    axes2[1].set_ylabel("Acceleration (m/s²)")
    plt.colorbar(h2[3], ax=axes2[1], label='Log Frequency')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizations Generated.")

In [ ]:
# CELL 11: Final Evaluation & JSON Save
print("\n=== FINAL EVALUATION ===")

overall = final_output["overall"]
failures = []

# Quality Threshold Logic
if overall.get('speed_difference_kmh', 0) > 5.0: # Example threshold
    failures.append(f"Speed diff {overall['speed_difference_kmh']:.2f} > 5.0 km/h")
    
if overall.get('vsp_rmse', 0) > 0.15: # Example threshold
    failures.append(f"VSP RMSE {overall['vsp_rmse']:.4f} > 0.15")

overall["validation_status"] = "FAILED" if failures else "PASSED"
overall["failures"] = failures

# Construct final JSON EXACTLY as requested
final_json_structure = {
    "overall": overall,
    "details": final_output["details"],
    "model_info": {
        "model_name": model_name,
        "run_timestamp": RUN_TIMESTAMP
    }
}

print(json.dumps(final_json_structure["overall"], indent=2))

# Save to MinIO
with fs.open(OUTPUT_METRICS_PATH, 'w') as f:
    json.dump(final_json_structure, f, indent=2)
    
print(f"✅ Metrics saved to {OUTPUT_METRICS_PATH}")

if failures:
    print("\n⚠️ VALIDATION FAILED")
    for f in failures: print(f" - {f}")
else:
    print("\n✅ VALIDATION PASSED")